In [110]:
import torch
import torchvision
import torchvision.datasets
import torchvision.transforms
import torch.utils.data
import torch.nn
import os
from matplotlib import pyplot as plot
import pickle

In [111]:
dir_name = os.getcwd()
batch_size = 100

def show_images(images, title):
  pass

train_dataset = torchvision.datasets.CIFAR10(root = dir_name, train = True, download = True,
                                           transform = torchvision.transforms.ToTensor())
test_dataset = torchvision.datasets.CIFAR10(root = dir_name, train = False, download = True,
                                          transform = torchvision.transforms.ToTensor())

#print('Number of train samples: {}'.format(len(train_dataset)))
#show_images(train_dataset, 'Train samples')

#print('Number of test samples: {}'.format(len(test_dataset)))
#show_images(test_dataset, 'Test samples')

train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size,
                                                shuffle = True)
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size = batch_size,
                                               shuffle = False)

In [112]:
class ConvolutionalNeuralNetwork(torch.nn.Module):
    def __init__(self):
        super(ConvolutionalNeuralNetwork, self).__init__()

        # Первый сверточный блок
        # 32 x ((32+2-3)/1+1) x ((32+2-3)/1+1) = 32 x 32 x 32
        self.conv1 = torch.nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.activation1 = torch.nn.ReLU()
        # 32 x ((32-2)/2+1) x ((32-2)/2+1) = 32 x 16 x 16
        self.pooling1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)

        # Второй сверточный блок
        # 64 x ((16+2-3)/1+1) x ((16+2-3)/1+1) = 64 x 16 x 16
        self.conv2 = torch.nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.activation2 = torch.nn.ReLU()
        # 64 x ((16-2)/2+1) x ((16-2)/2+1) = 64 x 8 x 8
        self.pooling2 = torch.nn.MaxPool2d(kernel_size=2, stride=2)

        # Третий сверточный блок
        # 128 x ((8+2-3)/1+1) x ((8+2-3)/1+1) = 128 x 8 x 8
        self.conv3 = torch.nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.activation3 = torch.nn.ReLU()
        # 128 x ((8-2)/2+1) x ((8-2)/2+1) = 128 x 4 x 4
        self.pooling3 = torch.nn.MaxPool2d(kernel_size=2, stride=2)

        # Полносвязные слои
        self.dropout1 = torch.nn.Dropout(0.25)
        self.linear1 = torch.nn.Linear(128 * 4 * 4, 256)
        self.activation4 = torch.nn.ReLU()
        self.dropout2 = torch.nn.Dropout(0.5)
        self.linear2 = torch.nn.Linear(256, 10)

    def forward(self, x):
        # Сверточные блоки
        out = self.conv1(x)
        out = self.activation1(out)
        out = self.pooling1(out)

        out = self.conv2(out)
        out = self.activation2(out)
        out = self.pooling2(out)

        out = self.conv3(out)
        out = self.activation3(out)
        out = self.pooling3(out)

        # Полносвязные слои
        out = out.view(out.size(0), -1)
        out = self.dropout1(out)
        out = self.linear1(out)
        out = self.activation4(out)
        out = self.dropout2(out)
        out = self.linear2(out)

        return out

cnn_model = ConvolutionalNeuralNetwork()

# Логирование информации о параметрах модели
#print('Parameters of convolutions:\n1. Kernels = {}\n2. Biases = {}'.
#      format(cnn_model.conv1.weight.shape, cnn_model.conv.bias.shape))
#print('Parameters of fully-connected layer:\n1. Weight matrix = {}\n2. Biases = {}'.
#      format(cnn_model.linear.weight.shape, cnn_model.linear.bias.shape))

In [113]:
learning_rate = 0.1
num_epochs = 10

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
cnn_model = cnn_model.to(device)
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(cnn_model.parameters(), lr = learning_rate)

# top-1
def get_accuracy(data_loader, model):
    tp = 0
    n = 0
    with torch.no_grad():
        for images, labels in data_loader:
            images = images.requires_grad_().to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            n += labels.size(0)
            tp += (predicted == labels).sum()
    return tp / n

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_data_loader):
        images = images.requires_grad_().to(device)
        labels = labels.to(device)
        outputs = cnn_model(images)
        loss = loss_function(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # Логирование метрики качества на тренировочных данных по завершении эпохи
    print('Epoch[{}]: accuracy = {}, loss: {}'.
          format(epoch, get_accuracy(train_data_loader, cnn_model), loss.item()))

Epoch[0]: accuracy = 0.3248800039291382, loss: 1.8341436386108398
Epoch[1]: accuracy = 0.44457998871803284, loss: 1.4906792640686035
Epoch[2]: accuracy = 0.5069599747657776, loss: 1.3035980463027954
Epoch[3]: accuracy = 0.5356599688529968, loss: 1.2573318481445312
Epoch[4]: accuracy = 0.5907399654388428, loss: 1.1468548774719238
Epoch[5]: accuracy = 0.5953599810600281, loss: 1.054322600364685
Epoch[6]: accuracy = 0.6668399572372437, loss: 0.9790959358215332
Epoch[7]: accuracy = 0.6775799989700317, loss: 1.0551016330718994
Epoch[8]: accuracy = 0.7069599628448486, loss: 1.0137357711791992
Epoch[9]: accuracy = 0.7288999557495117, loss: 0.7500267028808594


In [114]:
# Логирование метрики качества на тренировочных данных
print('Test accuracy: {}'.format(get_accuracy(test_data_loader, cnn_model)))

Test accuracy: 0.6805999875068665
